In [ ]:
!pip install pygam
!pip install scipy
!pip install shap xgboost==3.0.5

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.patches import Patch

### Gini

In [ ]:
socioeconomic_indicators = [
    "Total Population",
    "Per Capita Income",
    "Home Value"
]

# Gini
def calculate_gini_from_df(df, population_col, ugs_col):
    population = df[population_col].values
    ugs_area = df[ugs_col].values

    sorted_indices = np.argsort(ugs_area)
    sorted_population = population[sorted_indices]
    sorted_ugs_area = ugs_area[sorted_indices]

    cumulative_population = np.cumsum(sorted_population) / np.sum(population)
    cumulative_ugs = np.cumsum(sorted_ugs_area) / np.sum(ugs_area)

    gini = 1 - np.sum((cumulative_population[1:] - cumulative_population[:-1]) *
                      (cumulative_ugs[1:] + cumulative_ugs[:-1]))

    return gini, cumulative_population, cumulative_ugs

gini_results = {}
for city_name, df in city_data.items():
    gini_results[city_name] = {}
    for indicator in socioeconomic_indicators:
        if indicator in df.columns and 'Cumulative shades' in df.columns:
            gini, _, _ = calculate_gini_from_df(df, indicator, 'Cumulative shades')
            gini_results[city_name][indicator] = gini
        else:
            gini_results[city_name][indicator] = np.nan

# subplot
num_indicators = len(socioeconomic_indicators)
num_cols = 1
num_rows = int(np.ceil(num_indicators / num_cols))

fig, axes = plt.subplots(num_rows, num_cols, figsize=(6, 6 * num_rows))
axes = axes.reshape(num_rows, num_cols)

for i, indicator in enumerate(socioeconomic_indicators):
    gini_values = []
    city_names = []

    row = i // num_cols
    col = i % num_cols
    ax = axes[row, col]

    ax.plot([0, 1], [0, 1], linestyle="--", color="gray")

    for city_name, df in city_data.items():
        if indicator not in df.columns or 'Cumulative shades' not in df.columns:
            continue

        _, cumulative_population, cumulative_ugs = calculate_gini_from_df(df, indicator, 'Cumulative shades')
        ax.plot(cumulative_population, cumulative_ugs, color=city_colors[city_name], lw= 4)

        gini = gini_results[city_name][indicator]
        gini_values.append(gini)
        city_names.append(city_name)

    # inset
    inset_ax = ax.inset_axes([0.15, 0.65, 0.3, 0.3])
    x_pos = np.arange(len(city_names))
    bars = inset_ax.bar(x_pos, gini_values, color=[city_colors[name] for name in city_names])
    inset_ax.set_ylim(0, 1)
    inset_ax.set_ylabel("Gini", fontsize=12)
    inset_ax.set_xticks(x_pos)
    inset_ax.set_xticklabels(city_names, rotation=90, fontsize=10)
    inset_ax.tick_params(axis='y', labelsize=10)
    inset_ax.tick_params(axis='x', length=0)

    ax.set_title(f"{indicator}", fontsize=16)
    ax.set_xlabel("Cumulative Share", fontsize=16)
    ax.set_ylabel("Cumulative Shade Share", fontsize=16)
    ax.tick_params(axis='both', labelsize=16)
    ax.grid()

plt.tight_layout(rect=[0, 0.1, 1, 1])
plt.show()

In [ ]:
import numpy as np
import pandas as pd

# Bootstrap
def bootstrap_gini(df, population_col, ugs_col, n_bootstraps=500, ci=0.95):
    gini_bootstrap = []
    n = len(df)

    for _ in range(n_bootstraps):
        sample_df = df.sample(n=n, replace=True)
        gini, _, _ = calculate_gini_from_df(sample_df, population_col, ugs_col)
        gini_bootstrap.append(gini)

    gini_mean = np.mean(gini_bootstrap)
    gini_std = np.std(gini_bootstrap)
    lower = np.percentile(gini_bootstrap, (1 - ci) / 2 * 100)
    upper = np.percentile(gini_bootstrap, (1 + ci) / 2 * 100)

    return gini_mean, gini_std, lower, upper

results = []

for city_name, df in city_data.items():
    for indicator in socioeconomic_indicators:
        if indicator in df.columns and 'Cumulative shades' in df.columns:
            mean_gini, std_gini, lower_ci, upper_ci = bootstrap_gini(df, indicator, 'Cumulative shades', n_bootstraps=500)
            results.append({
                "City": city_name,
                "Indicator": indicator,
                "Gini": mean_gini,
                "Std": std_gini,
                "CI Lower": lower_ci,
                "CI Upper": upper_ci
            })

gini_ci_df = pd.DataFrame(results)
gini_ci_df = gini_ci_df.sort_values(by=["Indicator", "Gini"], ascending=[True, False]).reset_index(drop=True)

### XGBoost

In [5]:
import xgboost as xgb
import shap
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

In [ ]:
indicators = [
    "Total Population", "Home Value", "Per Capita Income", "Total Households",
    "Construction before 2000", "Construction after 2000",
    "Average Building Height", "Average Canopy Height"
]

fig, axs = plt.subplots(2, 4, figsize=(12, 6))
axs = axs.ravel()

all_shap_values = {i: [] for i in range(len(indicators))}
all_feature_values = {i: [] for i in range(len(indicators))}
all_colors = []

# sample weights
weights = {city: len(df) for city, df in city_data.items()}

for city_name, df in city_data.items():
    color = city_colors.get(city_name, 'gray')

    df_clean = df.dropna(subset=indicators + ['Average shades'])
    X_raw = df_clean[indicators].values
    y = df_clean['Average shades'].values

    # standerdize
    scaler = MinMaxScaler()
    X = scaler.fit_transform(X_raw)

    # XGBoost
    model = xgb.XGBRegressor(learning_rate=0.01, n_estimators=250, max_depth=7, subsample=0.6, colsample_bytree=0.6, random_state=42)
    model.fit(X, y, sample_weight=[weights[city_name]] * len(y))  # sample weight

    # SHAP
    explainer = shap.Explainer(model)
    shap_values = explainer(X)

    # SHAP dependence plot
    for i in range(len(indicators)):
        axs[i].scatter(
            X[:, i],
            shap_values[:, i].values,
            color=color,
            alpha=0.2,
            s=10,
            label=city_name if i == 0 else None
        )
        axs[i].set_title(indicators[i])
        axs[i].grid(True)

legend_elements = [Patch(facecolor=color, label=city) for city, color in city_colors.items()]
fig.legend(handles=legend_elements, loc='lower center', bbox_to_anchor=(0.5, -0.12),
           ncol=int(np.ceil(len(legend_elements) / 2)), fontsize=10, frameon=False)

plt.tight_layout()
plt.show()

In [ ]:
#tunning
fig, axs = plt.subplots(2, 4, figsize=(12, 6))
axs = axs.ravel()

all_shap_values = {i: [] for i in range(len(indicators))}
all_feature_values = {i: [] for i in range(len(indicators))}
all_colors = []

weights = {city: len(df) for city, df in city_data.items()}

# tuning network
param_grid = {
    'n_estimators': [100, 150, 200, 250, 300],
    'max_depth': [4, 5, 6, 7, 8, 9],
    'learning_rate': [0, 0.01, 0.03, 0.05],
    'subsample': [0.2, 0.4, 0.6, 0.8],
    'colsample_bytree': [0.2, 0.4, 0.6, 0.8],
}

model = xgb.XGBRegressor(random_state=42)

# grid serch and five fold
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1, verbose=1)
grid_search.fit(X, y)

print("Best parameters found: ", grid_search.best_params_)

best_model = grid_search.best_estimator_

cv_scores = cross_val_score(best_model, X, y, cv=5, scoring='neg_mean_squared_error')

print("Cross-validation MSE: ", -cv_scores.mean())
print("Cross-validation Std: ", cv_scores.std())

y_pred = best_model.predict(X)
print("R²:", r2_score(y, y_pred))
print("MAE:", mean_absolute_error(y, y_pred))
print("RMSE:", np.sqrt(mean_squared_error(y, y_pred)))

# SHAP
for city_name, df in city_data.items():
    color = city_colors.get(city_name, 'gray')

    df_clean = df.dropna(subset=indicators + ['Average shades'])
    X_raw = df_clean[indicators].values
    y = df_clean['Average shades'].values

    scaler = MinMaxScaler()
    X = scaler.fit_transform(X_raw)

    best_model.fit(X, y, sample_weight=[weights[city_name]] * len(y))

    explainer = shap.Explainer(best_model)
    shap_values = explainer(X)

    for i in range(len(indicators)):
        axs[i].scatter(
            X[:, i],
            shap_values[:, i].values,
            color=color,
            alpha=0.2,
            s=10,
            label=city_name if i == 0 else None
        )
        axs[i].set_title(indicators[i])
        axs[i].grid(True)

legend_elements = [Patch(facecolor=color, label=city) for city, color in city_colors.items()]
fig.legend(handles=legend_elements, loc='lower center', bbox_to_anchor=(0.5, -0.12),
           ncol=int(np.ceil(len(legend_elements) / 2)), fontsize=10, frameon=False)

plt.tight_layout()
plt.show()